Input midi file path, and extract melody instrument. 

In [ ]:

import pretty_midi

def get_midi_data(path):
    midi_data = pretty_midi.PrettyMIDI(path)
    return midi_data

def get_time_signiture(midi_data):
    time_signatures = midi_data.time_signature_changes
    return time_signatures

def get_melody_instrument(midi_data,show=False):
    melody_instrument = None
    highest_avg_pitch = -1
    min_overlap_count = float('inf')

    for instrument in midi_data.instruments:
        if instrument.is_drum:
            continue  

        if len(instrument.notes) > 0:
            avg_pitch = sum(note.pitch for note in instrument.notes) / len(instrument.notes)

            instrument.notes.sort(key=lambda x: x.start)
            overlap_count = 0
            for i in range(1, len(instrument.notes)):
                if instrument.notes[i].start < instrument.notes[i - 1].end:
                    overlap_count += 1

            pitch_values = [note.pitch for note in instrument.notes]
            pitch_variance = max(pitch_values) - min(pitch_values) if len(pitch_values) > 1 else 0

            if (avg_pitch > highest_avg_pitch and 
                overlap_count < min_overlap_count and 
                pitch_variance > 0):
                highest_avg_pitch = avg_pitch
                min_overlap_count = overlap_count
                melody_instrument = instrument
    if show:
        if melody_instrument:
            melody_notes = melody_instrument.notes
            for note in melody_notes:
                print(f"Note: {pretty_midi.note_number_to_name(note.pitch)}, "
                    f"Start: {note.start:.2f}s, End: {note.end:.2f}s, "
                    f"Velocity: {note.velocity}")
        else:
            print("No melody instrument detected.")
        
    melody_notes = melody_instrument.notes
    return melody_instrument,melody_notes
corresponding_beat_path='/none_beat_midi.txt' # or the beat information file of the midi file, if applicable
midi_data=get_midi_data('your_own_path')
melody_instrument,melody_notes = get_melody_instrument(midi_data,show=True)





Combine the rest tracks (except drum and pedal) into one track as accompaniment instrument. 

In [ ]:
def get_accompaniment_instrument(midi_data,show=False):
    accompaniment_midi = pretty_midi.PrettyMIDI()
    accompaniment_notes = []
    accompaniment_instrument = pretty_midi.Instrument(program=0, name="Accompaniment Track")

    melody_instrument,_=get_melody_instrument(midi_data,show=False)
    for i, instrument in enumerate(midi_data.instruments):

        if instrument == melody_instrument or instrument.is_drum:
            continue

        if 'pedal' in instrument.name.lower():
            continue

        accompaniment_instrument.notes.extend(instrument.notes)
        accompaniment_notes.extend(instrument.notes)

    accompaniment_midi.instruments.append(accompaniment_instrument)
    accompaniment_notes.sort(key=lambda x: x.start)
    if show:
        for note in accompaniment_notes:
            print(f"Note: {pretty_midi.note_number_to_name(note.pitch)}, "
                f"Start: {note.start:.2f}s, End: {note.end:.2f}s, "
                f"Velocity: {note.velocity}")
    return accompaniment_notes


accompaniment_instrument= get_accompaniment_instrument(midi_data,show=True)

Caculate the beats. 

In [ ]:

def calculate_positions(times, speeds, step=0.001,endtime=10):
    positions = []
    current_position = 0
    current_speed_index = 0
    current_speed = speeds[current_speed_index]
    max_time = endtime  

    for t in range(int(max_time / step) + 1):
        time_in_seconds = t * step
        if current_speed_index < len(times) - 1 and time_in_seconds >= times[current_speed_index + 1]:
            current_speed_index += 1
            current_speed = speeds[current_speed_index]
        
        current_position += current_speed/60 * step
        positions.append(current_position)
    return positions

def find_time_for_each_x(positions, x, step=0.01):
    distances = []
    targets = x
    for i, position in enumerate(positions):
        if position >= targets:
            distances.append(i * step)
            targets += x
    return distances

def parse_beats(beat_file_path):
    with open(beat_file_path, 'r') as file:
        lines = file.readlines()

    start_times = [float(line.split()[0]) for line in lines]

    end_times = [start_times[i+1] for i in range(len(start_times) - 1)]

    if len(start_times) > 1:
        end_times.append(start_times[-1] + (start_times[-1] - start_times[-2]))
    elif len(start_times) == 1:
        end_times.append(start_times[0] + 1.0)  
    return end_times


def get_beat_time_table(midi_file,backup_beat_path,show=False):
    tempo_times, tempo_changes = midi_data.get_tempo_changes()
    end_time=midi_data.get_end_time()
    time_signature_changes = midi_data.time_signature_changes

    try:
        if show:
            print('trying to find existing txt file')
        final_beats_time_table=parse_beats(backup_beat_path)
    except Exception:
        final_beats_time_table=[]
        if show:
            print('using data detected by pretty midi')
        current_index = 0
        while current_index < len(time_signature_changes):
            current_time_signature = time_signature_changes[current_index]
            if current_index + 1 < len(time_signature_changes):
                next_time_signature = time_signature_changes[current_index + 1]
                segment_duration = next_time_signature.time - current_time_signature.time
            else:
                segment_duration = midi_data.get_end_time() - current_time_signature.time

            beats_per_bar = current_time_signature.numerator
            dt=0.0001
            truth_table=calculate_positions(tempo_times,tempo_changes,step=dt,endtime=end_time)
            beats_time_table=find_time_for_each_x(truth_table,x=1,step=dt)
            finish_time=midi_data.get_end_time()
            
            for i in beats_time_table:
                if i <=finish_time:
                    final_beats_time_table.append(i)
            current_index+=1

    if show:
        for i, timestamp in enumerate(final_beats_time_table):
            print(f"beat {i + 1} starts at: {timestamp:.4f} seconds")
    return final_beats_time_table

beat_time_table=get_beat_time_table(midi_data,corresponding_beat_path,show=True)

Get all the melody notes and accompaniment notes in each beat.

In [ ]:
def get_notes_by_beat(acmt,beats,show=False):
    beats_durations=[]
    for i in range(0,len(beats)):
        if i==0:
            beats_durations.append([0,beats[i]])
        else:
            beats_durations.append([beats[i-1],beats[i]])

    acmt_by_beats=[]
    for i in range(0,len(beats_durations)):
        detected_acmt_notes=[]

        for j in range(0,len(acmt)): 
            if acmt[j].end<beats_durations[i][0] or acmt[j].start>beats_durations[i][1]:
                pass
            else:
                detected_acmt_notes.append(pretty_midi.note_number_to_name(acmt[j].pitch))

        one_record={
            'beat_index':int(i+1),
            'start':beats_durations[i][0],
            'end':beats_durations[i][1],
            'detected_notes': detected_acmt_notes
        }
        acmt_by_beats.append(one_record)
    if show:
        for i in acmt_by_beats :
            print(f'beat index {i['beat_index']},{i['detected_notes']},start at {i['start']:.2f}, end at {i['end']:.2f}')

    return acmt_by_beats

acmt_note_by_beat=get_notes_by_beat(accompaniment_instrument,beat_time_table,show=True)

melody_notes_by_beat=get_notes_by_beat(melody_notes,beat_time_table,show=True)


Pair each note in the melody with all the notes in the accompaniment in the beat to get all the intervals obtained in the beat.

In [ ]:
def get_pairs(a,b):
    pairs=[]
    if a !=[] and b !=[]:
        for i in a: 
            for j in b:
                the_pair=[i,j]
                pairs.append(the_pair)
    return pairs

def get_pairs_by_beat(melody,acmt,beats,show=False):
    all_beats=[]
    for i in range(0,len(beats)):
        melody_notes_in_this_beat=melody[i]['detected_notes']
        acmt_notes_in_this_beat=acmt[i]['detected_notes']
        the_pairs=get_pairs(melody_notes_in_this_beat,acmt_notes_in_this_beat)
        the_start_time=melody[i]['start']
        the_end_time=melody[i]['end']
        the_index=i+1

        one_record={
            'beat_index':the_index,
            'start':the_start_time,
            'end':the_end_time,
            'the_pairs':the_pairs,
        }
        all_beats.append(one_record)
    if show:
        for i in all_beats:
            print(f'beat index {i['beat_index']},{i['the_pairs']},start at {i['start']:.2f}, end at {i['end']:.2f}')
    return all_beats


pairs_by_beat=get_pairs_by_beat(melody_notes_by_beat,acmt_note_by_beat,beat_time_table,show=True)

Intervals and corresponding score. 

In [ ]:
def get_interval_and_scores():
    chromatic_scale = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]

    intervals_info = { 
        "Perfect Unison": 0,
        "Minor Second": 70,
        "Major Second": 60,
        "Minor Third": 12.5,
        "Major Third": 10,
        "Perfect Fourth": 5,
        "Tritone": 50,
        "Perfect Fifth": 2.5,
        "Minor Sixth": 17.5,
        "Major Sixth": 15,
        "Minor Seventh": 55,
        "Major Seventh": 65,
        "Perfect Octave": 0
    }

    interval_dict = {}
    for root in chromatic_scale:
        root_index = chromatic_scale.index(root)
        for i, interval_name in enumerate(intervals_info.keys()):
            note_index = (root_index + i) % 12  
            pair = f"{root} to {chromatic_scale[note_index]}"
            interval_dict[pair] = intervals_info[interval_name]
    return interval_dict

all_intervals_categorized = get_interval_and_scores()

Get final score and other information. 

In [ ]:
def judge(detected_pairs,interval_categorized,show=False):
    counts_perfect=0
    counts_dis=0
    counts_imperfect=0
    all_intervals_categorized=interval_categorized

    interval_score_board={}
    for i in all_intervals_categorized.keys():
        interval_score_board[i]=0

    for i in detected_pairs:
        the_pairs=i['the_pairs']
        for j in the_pairs:
            x=j[0]
            y=j[1]
            op=f'{x[:-1]} to {y[:-1]}'
        

            if op in all_intervals_categorized.keys():
                if all_intervals_categorized[op]<=5 :      
                    counts_perfect+=1
                elif all_intervals_categorized[op]<=17.5:  
                    counts_imperfect+=1
                elif all_intervals_categorized[op]<=70:            
                    counts_dis+=1

            if op in all_intervals_categorized.keys():
                interval_score_board[op]=interval_score_board[op]+all_intervals_categorized[op]
    sum_counts=counts_imperfect+counts_dis+counts_perfect
    score=0
    for op in interval_categorized.keys():
        score+=interval_score_board[op]
    if sum_counts==0:
        perfect_ratio=0
        imperfect_ratio=0
        dis_ratio=0
        score_perbeat=0
    else:
        perfect_ratio=counts_perfect/sum_counts*100
        imperfect_ratio=counts_imperfect/sum_counts*100
        dis_ratio=counts_dis/sum_counts*100
        score_perbeat=score/sum_counts


    if show:
        print(counts_perfect,counts_imperfect,counts_dis)
        print('badness per beat:')
        print(score_perbeat)
        print(perfect_ratio,imperfect_ratio,dis_ratio)

    return [counts_perfect,counts_imperfect,counts_dis,sum_counts,score_perbeat,perfect_ratio,imperfect_ratio,dis_ratio]

final_report=judge(pairs_by_beat,all_intervals_categorized,show=True)